# comercio_ext_auxiliares.tb_auxiliar_recintos_alfandegarios
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/URF_delta`
## 📌 Descrição do arquivo
Referência de **URF** (recintos alfandegários).
|Coluna|Descrição|
|---|---|
|`CO_URF`|Código URF|
|`NO_URF`|Nome URF|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
# ---------------------------------------
# comercio_ext_auxiliares.tb_auxiliar_recintos_alfandegarios
# ---------------------------------------














In [0]:
from pyspark.sql import functions as F, types as T
from delta.tables import DeltaTable
def up(c): return F.upper(F.trim(F.col(c).cast(T.StringType())))

In [0]:
bronzePath  = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/urf/"
silverTable = "silver_comercio_ext_auxiliares.tb_auxiliar_recintos_alfandegarios"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_URF",   T.StringType(),   False),
    T.StructField("NO_URF",   T.StringType(),   False),
    T.StructField("TS_REF",   T.TimestampType(),False),
    T.StructField("NM_ORIGEM",T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
df_norm = (
    df_bronze
    .withColumn("CO_URF", up("CO_URF"))
    .withColumn("NO_URF", F.col("NO_URF").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/URF_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_URF").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_URF"])

In [0]:
df_silver = df_dedup.select("CO_URF","NO_URF","TS_REF","NM_ORIGEM")

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, silverTable)
merge_condition = "t.CO_URF = s.CO_URF"

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "NO_URF":   "s.NO_URF",
        "TS_REF":   "s.TS_REF",
        "NM_ORIGEM":"s.NM_ORIGEM",
    })
    .whenNotMatchedInsertAll()
    .execute())